In [136]:
%matplotlib qt5
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import sunpy.io.fits
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MultipleLocator

In [137]:
base_path = Path('/home/harsh/SpinorNagaraju/maps_1/stic/processed_inputs/')

In [138]:
filename = 'alignedspectra_scan1_map01_Ca.fits_stic_profiles.nc'

In [139]:
f = h5py.File(base_path / filename, 'r')
ind = np.where(f['profiles'][0, 0, 0, :, 0] != 0)[0]
wave = f['wav'][ind]
data = f['profiles'][()][0, :, :, ind, 0]
f.close()

In [140]:
data = data.reshape(306, 19 * 60).T

In [166]:
profile_ind = 0
pixel_cat_arr = np.ones((19 * 60), dtype=np.int64) * -1
cat_data = np.zeros((10, 306), dtype=np.float64)
write_path = Path('/home/harsh/SpinorNagaraju/maps_1/stic/manual_categories/')
filename = 'manual_labels.txt'

fig = plt.figure(figsize=(19, 19 * 9/16))
gs1 = GridSpec(2, 5)
gs1.update(left=0.05, bottom=0.4, right=0.99, top=0.95)
gs2 = GridSpec(1, 1)
gs2.update(left=0.05, bottom=0.05, right=0.5, top=0.35)

axs = list()
for k in range(10):
    axs.append(fig.add_subplot(gs1[k]))
axs.append(fig.add_subplot(gs2[0]))

previous_axs = plt.axes([0.53, 0.22, 0.05, 0.05])
next_axs = plt.axes([0.59, 0.22, 0.05, 0.05])
cat_1_axs = plt.axes([0.65, 0.22, 0.05, 0.05])
cat_2_axs = plt.axes([0.71, 0.22, 0.05, 0.05])
cat_3_axs = plt.axes([0.77, 0.22, 0.05, 0.05])
cat_4_axs = plt.axes([0.83, 0.22, 0.05, 0.05])
cat_5_axs = plt.axes([0.89, 0.22, 0.05, 0.05])
cat_6_axs = plt.axes([0.65, 0.15, 0.05, 0.05])
cat_7_axs = plt.axes([0.71, 0.15, 0.05, 0.05])
cat_8_axs = plt.axes([0.77, 0.15, 0.05, 0.05])
cat_9_axs = plt.axes([0.83, 0.15, 0.05, 0.05])
cat_10_axs = plt.axes([0.89, 0.15, 0.05, 0.05])
reset_axs = plt.axes([0.53, 0.15, 0.05, 0.05])
load_axs = plt.axes([0.59, 0.15, 0.05, 0.05])

prev_btn = Button(previous_axs, 'Previous')
next_btn = Button(next_axs, 'Next')
cat_1_btn = Button(cat_1_axs, 'Cat 1')
cat_2_btn = Button(cat_2_axs, 'Cat 2')
cat_3_btn = Button(cat_3_axs, 'Cat 3')
cat_4_btn = Button(cat_4_axs, 'Cat 4')
cat_5_btn = Button(cat_5_axs, 'Cat 5')
cat_6_btn = Button(cat_6_axs, 'Cat 6')
cat_7_btn = Button(cat_7_axs, 'Cat 7')
cat_8_btn = Button(cat_8_axs, 'Cat 8')
cat_9_btn = Button(cat_9_axs, 'Cat 9')
cat_10_btn = Button(cat_10_axs, 'Cat 10')
reset_btn = Button(reset_axs, 'Reset')
load_btn = Button(load_axs, 'Load')

im0, = axs[0].plot(np.zeros(306), color='blue')
im1, = axs[1].plot(np.zeros(306), color='blue')
im2, = axs[2].plot(np.zeros(306), color='blue')
im3, = axs[3].plot(np.zeros(306), color='blue')
im4, = axs[4].plot(np.zeros(306), color='blue')
im5, = axs[5].plot(np.zeros(306), color='blue')
im6, = axs[6].plot(np.zeros(306), color='blue')
im7, = axs[7].plot(np.zeros(306), color='blue')
im8, = axs[8].plot(np.zeros(306), color='blue')
im9, = axs[9].plot(np.zeros(306), color='blue')
im10, = axs[10].plot(data[profile_ind], color='blue')

title = axs[8].text(
    0.5, -0.4,
    'Profile: {}'.format(profile_ind),
    transform=axs[8].transAxes
)

def recalculate_categories():
    global pixel_cat_arr, cat_data
    for i in range(10):
        a = np.where(pixel_cat_arr == i)[0]
        if a.size <= 0:
            cat_data[i] = np.zeros(306)
            return
        cat_data[i] = np.mean(data[a], 0)

def update_cat_1(*args, **kwargs):
    global pixel_cat_arr, cat_data
    pixel_cat_arr[profile_ind] = 0
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()
    
def update_cat_2(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 1
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_3(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 2
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_4(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 3
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_5(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 4
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_6(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 5
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_7(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 6
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_8(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 7
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_9(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 8
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_10(*args, **kwargs):
    global pixel_cat_arr
    pixel_cat_arr[profile_ind] = 9
    recalculate_categories()
    update_cat_plots()
    set_color()
    save_data()
    fig.canvas.draw_idle()

def update_cat_plots():
    global cat_data
    im0.set_ydata(cat_data[0])
    im1.set_ydata(cat_data[1])
    im2.set_ydata(cat_data[2])
    im3.set_ydata(cat_data[3])
    im4.set_ydata(cat_data[4])
    im5.set_ydata(cat_data[5])
    im6.set_ydata(cat_data[6])
    im7.set_ydata(cat_data[7])
    im8.set_ydata(cat_data[8])
    im9.set_ydata(cat_data[9])
    axs[0].set_ylim(cat_data[0].min() * 0.95,cat_data[0].max() * 1.05)
    axs[1].set_ylim(cat_data[1].min() * 0.95,cat_data[1].max() * 1.05)
    axs[2].set_ylim(cat_data[2].min() * 0.95,cat_data[2].max() * 1.05)
    axs[3].set_ylim(cat_data[3].min() * 0.95,cat_data[3].max() * 1.05)
    axs[4].set_ylim(cat_data[4].min() * 0.95,cat_data[4].max() * 1.05)
    axs[5].set_ylim(cat_data[5].min() * 0.95,cat_data[5].max() * 1.05)
    axs[6].set_ylim(cat_data[6].min() * 0.95,cat_data[6].max() * 1.05)
    axs[7].set_ylim(cat_data[7].min() * 0.95,cat_data[7].max() * 1.05)
    axs[8].set_ylim(cat_data[8].min() * 0.95,cat_data[8].max() * 1.05)
    axs[9].set_ylim(cat_data[9].min() * 0.95,cat_data[9].max() * 1.05)

def set_color():
    im0.set_color('blue')
    im1.set_color('blue')
    im2.set_color('blue')
    im3.set_color('blue')
    im4.set_color('blue')
    im5.set_color('blue')
    im6.set_color('blue')
    im7.set_color('blue')
    im8.set_color('blue')
    im9.set_color('blue')
    if pixel_cat_arr[profile_ind] == 0:
        im0.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 1:
        im1.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 2:
        im2.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 3:
        im3.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 4:
        im4.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 5:
        im5.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 6:
        im6.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 7:
        im7.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 8:
        im8.set_color('darkgreen')
    if pixel_cat_arr[profile_ind] == 9:
        im9.set_color('darkgreen')

def load_cat_from_file(*args, **kwargs):
    global pixel_cat_arr, cat_data
    pixel_cat_arr = np.loadtxt(write_path / filename).astype(np.int64)
    recalculate_categories()
    update_cat_plots()
    set_color()
    fig.canvas.draw_idle()

def update_prev(*args, **kwargs):
    global profile_ind
    if profile_ind == 0:
        return
    profile_ind -= 1
    im10.set_ydata(data[profile_ind])
    axs[10].set_ylim(data[profile_ind].min() * 0.95, data[profile_ind].max() * 1.05)
    title.set_text('Profile: {}'.format(profile_ind))
    set_color()
    fig.canvas.draw_idle()

def update_next(*args, **kwargs):
    global profile_ind
    total = 19 * 60 - 1
    if profile_ind == total:
        return
    profile_ind += 1
    im10.set_ydata(data[profile_ind])
    axs[10].set_ylim(data[profile_ind].min() * 0.95, data[profile_ind].max() * 1.05)
    title.set_text('Profile: {}'.format(profile_ind))
    set_color()
    fig.canvas.draw_idle()

def save_data(*args, **kwargs):
    global pixel_cat_arr
    np.savetxt(write_path / filename, pixel_cat_arr)

def reset_data(*args, **kwargs):
    global pixel_cat_arr, cat_data
    pixel_cat_arr = np.ones((19 * 60), dtype=np.int64) * -1
    cat_data = np.zeros((10, 306), dtype=np.float64)
    update_cat_plots()
    set_color()
    save_data()

prev_btn.on_clicked(update_prev)
next_btn.on_clicked(update_next)
cat_1_btn.on_clicked(update_cat_1)
cat_2_btn.on_clicked(update_cat_2)
cat_3_btn.on_clicked(update_cat_3)
cat_4_btn.on_clicked(update_cat_4)
cat_5_btn.on_clicked(update_cat_5)
cat_6_btn.on_clicked(update_cat_6)
cat_7_btn.on_clicked(update_cat_7)
cat_8_btn.on_clicked(update_cat_8)
cat_9_btn.on_clicked(update_cat_9)
cat_10_btn.on_clicked(update_cat_10)
load_btn.on_clicked(load_cat_from_file)
reset_btn.on_clicked(reset_data)

0

/home/harsh/CourseworkRepo/HMIAnalysis/env/lib/python3.7/site-packages/ipykernel_launcher.py:177: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/home/harsh/CourseworkRepo/HMIAnalysis/env/lib/python3.7/site-packages/ipykernel_launcher.py:178: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/home/harsh/CourseworkRepo/HMIAnalysis/env/lib/python3.7/site-packages/ipykernel_launcher.py:179: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/home/harsh/CourseworkRepo/HMIAnalysis/env/lib/python3.7/site-packages/ipykernel_launcher.py:180: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
/home/harsh/CourseworkRepo/HMIAnalysis/env/lib/python3.7/site-packages/ipykernel_launcher.py:181: UserWarning: A